In [1]:
import os
import re
import nltk
import shutil
from nltk.corpus import stopwords
from zipfile import ZipFile, BadZipFile

# Télécharger les stopwords une fois
nltk.download('stopwords')

# Obtenir les stopwords en français et anglais
stop_words_fr = set(stopwords.words('french'))
stop_words_en = set(stopwords.words('english'))

def clean_srt(input_file, output_file, encoding='utf-8'):
    try:
        with open(input_file, 'r', encoding=encoding) as file:
            lines = file.readlines()
        
        cleaned_lines = []
        
        for line in lines:
            if not line.strip().isdigit() and '-->' not in line:
                words = line.strip().split()
                filtered_words = [word for word in words if word.lower() not in stop_words_fr and word.lower() not in stop_words_en]
                cleaned_lines.append(' '.join(filtered_words))
        
        with open(output_file, 'w', encoding=encoding) as file:
            file.write('\n'.join(cleaned_lines))
    except UnicodeDecodeError:
        print(f"Erreur de décodage avec l'encodage {encoding}. Essayez un autre encodage.")

def extract_archive(archive_path, extract_to):
    if archive_path.endswith('.zip'):
        try:
            with ZipFile(archive_path, 'r') as zip_ref:
                zip_ref.extractall(extract_to)
        except BadZipFile:
            print(f"Erreur: {archive_path} n'est pas un fichier zip fonctionnel.")
    # Ajouter d'autres formats d'archive si nécessaire

def process_all_srt_files(root_folder, encoding='utf-8'):
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.srt'):
                input_file = file_path
                output_file = os.path.join(subdir, file.replace('.srt', '_cleaned.txt'))
                clean_srt(input_file, output_file, encoding)
            elif file.endswith('.zip'):
                extract_to = os.path.join(subdir, file.replace('.zip', ''))
                os.makedirs(extract_to, exist_ok=True)
                extract_archive(file_path, extract_to)
                process_all_srt_files(extract_to, encoding)

# Utilisation de la fonction pour traiter tous les fichiers .srt dans le dossier racine
root_folder = './subs'
process_all_srt_files(root_folder, encoding='latin1')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Etudiant\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Erreur: ./subs\blade\blades01VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\entourage\entouragesallVF\Entourage.502.VO.VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\entourage\entouragesallVO\Entourage.502.VO.VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\prisonbreak\prisonbreaks01VO.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\robinhood\robinhoods01s02VO\Robin.Hood.S1.DVDRip.VO.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\xfiles\xfiless01VF.zip n'est pas un fichier zip fonctionnel.
Erreur: ./subs\xfiles\xfiless03VF.zip n'est pas un fichier zip fonctionnel.


In [4]:
# Supprime les fichiers .srt, .zip et les dossiers vides
def delete_srt_files_and_empty_dirs(root_folder):
    # Parcourir tous les fichiers et dossiers à partir du dossier racine
    for subdir, _, files in os.walk(root_folder):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file.endswith('.srt') or file.endswith('.zip'):
                try:
                    os.remove(file_path)  # Supprimer le fichier .srt ou .zip
                except PermissionError:
                    print(f"Erreur de permission: Impossible de supprimer {file_path}")

    # Supprimer les dossiers vides
    for subdir, dirs, _ in os.walk(root_folder, topdown=False):
        for dir in dirs:
            dir_path = os.path.join(subdir, dir)
            try:
                os.rmdir(dir_path)  # Supprimer le dossier s'il est vide
            except OSError:
                pass  # Ignorer l'erreur si le dossier n'est pas vide

# Utilisation de la fonction pour supprimer les fichiers .srt, .zip et les dossiers vides dans le dossier racine
root_folder = './subs'  # Remplacez par le chemin de votre dossier racine
delete_srt_files_and_empty_dirs(root_folder)